In [1]:
import pickle 
import torch
import numpy as np
import pandas as pd
from model import *
from datagen.genset import GenSet
from training_helpers import *

torch.multiprocessing.set_start_method('spawn')
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)
torch.set_default_dtype(torch.float32)

In [2]:
train = GenSet()
valid = pickle.load("./valid/valid_data.pkl")

TypeError: file must have 'read' and 'readline' attributes

In [3]:
kwargs = dict(wavelet="bior2.2", samples=512, out=4, learnsamples=True)
model = WCNFourierModel(**kwargs).to(device)
model_pars = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_pars])

for p in model.named_parameters():
    print(p[0], np.prod(p[1].size()))
print(params)

cnn1.weight 624
cnn1.bias 16
cnn2.weight 4224
cnn2.bias 24
cnn3.weight 864
cnn3.bias 4
cnn4.weight 3168
cnn4.bias 32
cnn5.weight 3584
cnn5.bias 16
cnn6.weight 320
cnn6.bias 4
cnnafter1.weight 672
cnnafter1.bias 24
cnnafter2.weight 4896
cnnafter2.bias 12
cnnafter3.weight 132
cnnafter3.bias 1
layerone.weight 131072
layerone.bias 256
layertwo.weight 32768
layertwo.bias 128
layerthree.weight 512
layerthree.bias 4
fdft.freqs 512
norm.weight 128
norm.bias 128
bnorm.weight 3
bnorm.bias 3
184131


In [ ]:
ITERATIONS_PER_EPOCH = 100

optim = torch.optim.AdamW(model.parameters(), lr=0.000085, weight_decay=10**-5.5, amsgrad=True)
loss = torch.nn.CrossEntropyLoss().to(device)
epoch_training_losses = []
epoch_validation = []

for batch, label in train:
    epoch_training_loss = []
    optim.zero_grad()
    out = model(batch)
    loss = loss(out, label)
    loss.backward()
    optim.step()

    epoch_training_loss.append(loss.item())

    if len(epoch_training_losses) % ITERATIONS_PER_EPOCH == 0:
        model.eval()
        data, label, types = valid
        out = model(data)

        true_y = torch.argmax(label, dim=1).flatten().cpu().numpy()
        pred_y = torch.argmax(out, dim=1).flatten().cpu().numpy()

        epoch_training_losses.append(np.mean(epoch_training_loss))
        epoch_validation.append((types, pred_y))

        training_plot(epoch_training_losses, epoch_validation)

